In [1]:
#coding=utf-8
import pandas
import numpy as np
import gensim
import csv
from keras.preprocessing import sequence
dataset=[]
sent=[]
ne_tags=set()
with open('ner_dataset.csv',newline='\n') as csvfile:
    lines = csv.reader(csvfile,delimiter=',',quotechar='"')
    for item in lines:
        ne_tags.add(item[-1].split('-')[-1])
        if item[0]:
            if len(sent):
                dataset.append(sent)
            sent=[item[1:]]
        else:
            sent.append(item[1:])
print(len(dataset))
print(ne_tags)

Using TensorFlow backend.


47958
{'nat', 'geo', 'art', 'gpe', 'eve', 'tim', 'per', 'O', 'org'}


In [2]:
max_len=max([len(sent) for sent in dataset])
dataset_size = len(dataset)
indices = np.random.permutation(dataset_size)
train_ind, test_ind = indices[:int(dataset_size*0.8)], indices[int(dataset_size*0.8):]
train_set,test_set=[],[]
for ind in train_ind:
    train_set.append(dataset[ind])
for ind in test_ind:
    test_set.append(dataset[ind])
train_sents=[[t[0] for t in sent] for sent in train_set]
train_pos=[[t[1] for t in sent] for sent in train_set]
train_ne=[[t[2].split('-')[-1] for t in sent] for sent in train_set]
test_sents=[[t[0] for t in sent] for sent in test_set]
test_pos=[[t[1] for t in sent] for sent in test_set]
test_ne=[[t[2].split('-')[-1] for t in sent] for sent in test_set]
print(len(test_set),len(train_set))

9592 38366


In [3]:
def one_hot(labels,label='O'):
    vec=[0]*len(labels)
    vec[list(labels).index(label)]=1
    return np.array(vec)
lmodel=gensim.models.Word2Vec(train_sents,size=100)
train_x,train_y=[],[]
for i,sent in enumerate(train_sents):
    sent_len=len(sent)
    ele_vec=[np.zeros(100)]*(max_len-sent_len)
#     print(np.array(ele_vec).shape)
    ele_ne=[one_hot(ne_tags)]*(max_len-sent_len)
    for j,word in enumerate(sent):
        try:
            ele_vec.append(lmodel.wv[word])
            ele_ne.append(one_hot(ne_tags,train_ne[i][j]))
        except:
            new_wv = 2*np.random.randn(100)-1 # sample from normal distn
            norm_const = np.linalg.norm(new_wv)
            new_wv /= norm_const
            ele_vec.append(new_wv)
            ele_ne.append(one_hot(ne_tags))
    train_x.append(ele_vec)
    train_y.append(ele_ne)
train_x=np.array(train_x)
train_y=np.array(train_y)
print(train_x.shape,train_y.shape)

(38366, 104, 100) (38366, 104, 9)


In [4]:
test_x,test_y=[],[]
for i,sent in enumerate(test_sents):
    sent_len=len(sent)
    ele_vec=[np.zeros(100)]*(max_len-sent_len)
#     print(np.array(ele_vec).shape)
    ele_ne=[one_hot(ne_tags)]*(max_len-sent_len)
    for j,word in enumerate(sent):
        try:
            ele_vec.append(lmodel.wv[word])
            ele_ne.append(one_hot(ne_tags,test_ne[i][j]))
        except:
            new_wv = 2*np.random.randn(100)-1 # sample from normal distn
            norm_const = np.linalg.norm(new_wv)
            new_wv /= norm_const
            ele_vec.append(new_wv)
            ele_ne.append(one_hot(ne_tags))
    test_x.append(ele_vec)
    test_y.append(ele_ne)
test_x=np.array(test_x)
test_y=np.array(test_y)
print(test_x.shape,test_y.shape)

(9592, 104, 100) (9592, 104, 9)


In [5]:
from keras.preprocessing import sequence
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers.core import Activation
from keras.regularizers import l2
from keras.layers.wrappers import TimeDistributed,Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout

In [6]:
ner_model=Sequential()
ner_model.add(Bidirectional(LSTM(150,return_sequences=True,
                                 bias_regularizer=l2(0),
                                 activity_regularizer=l2(0.),
                                 kernel_regularizer=l2(0.))
                            ,
                            input_shape=(104,100)
                           ))
ner_model.add(Dropout(0.5))
ner_model.add(TimeDistributed(Dense(9,activation='softmax',
                                    bias_regularizer=l2(0.),
                                   activity_regularizer=l2(0.),
                                   kernel_regularizer=l2(0.))))
ner_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
ner_model.fit(train_x,train_y,epochs=5,batch_size=200)

Epoch 1/5
38366/38366 [==============================] - 412s - loss: 0.1862 - acc: 0.9716   
Epoch 2/5
38366/38366 [==============================] - 436s - loss: 0.0520 - acc: 0.9850   
Epoch 3/5
38366/38366 [==============================] - 448s - loss: 0.0430 - acc: 0.9873   
Epoch 4/5
38366/38366 [==============================] - 464s - loss: 0.0386 - acc: 0.9886   
Epoch 5/5
38366/38366 [==============================] - 465s - loss: 0.0357 - acc: 0.9894   


In [9]:
scores = ner_model.evaluate(test_x, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 99.06%


In [36]:
sentence='International Atomic Energy Agency is a place in America , while World Trade Organization is a place in Canada'
sentence=sentence.split()
sent_len=len(sentence)
ele_vec=[np.zeros(100)]*(max_len-sent_len)
for word in sentence:
    try:
        ele_vec.append(lmodel.wv[word])
    except:
        new_wv = 2*np.random.randn(100)-1 # sample from normal distn
        norm_const = np.linalg.norm(new_wv)
        new_wv /= norm_const
        ele_vec.append(new_wv)
X=np.array(ele_vec).reshape((1,max_len,100))
score = ner_model.predict(X,batch_size=1)
# print(score[0][-sent_len:][:])
res=[]
for i,s in enumerate(score[0][-sent_len:]):
    res.append((sentence[i],list(ne_tags)[np.argmax(s)]))
print(res)

[('International', 'org'), ('Atomic', 'org'), ('Energy', 'org'), ('Agency', 'org'), ('is', 'O'), ('a', 'O'), ('place', 'O'), ('in', 'O'), ('America', 'geo'), (',', 'O'), ('while', 'O'), ('World', 'O'), ('Trade', 'org'), ('Organization', 'org'), ('is', 'O'), ('a', 'O'), ('place', 'O'), ('in', 'O'), ('Canada', 'geo')]
